#### Due to confidentiality，the notebook only contains some parts of codes

In [ ]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.datasets import dump_svmlight_file
from sklearn.metrics import precision_score
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve

In [ ]:
rp = pd.read_csv('path//RP_CLEANED.csv')
print(rp.shape)
rp.head()

In [ ]:
# Defining a Function to calculate the missing values by column
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()

        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)

        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})

        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

        # Print some summary information
        print ("Dataframe Rows =  " + str(df.shape[0]) + "\n" + "Dataframe Columns =  " + str(df.shape[1]) + "\n" +
            "Columns with Null Values =  " + str(mis_val_table_ren_columns.shape[0]))

        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
# Get % of missing values for the columns with missing values.
missing_values_table(rp)

### Data wrangling

In [ ]:
# Change data type to datetime for further processing
rp['REGISTRATION_DATE'] = pd.to_datetime(rp['REGISTRATION_DATE'], format='%Y-%m-%d %H:%M:%S')

# Make sure all data are from 2016 to 2020
rp = rp[rp['REGISTRATION_DATE'].map(lambda x: x > datetime.date(2015,12,31))]

# reorganize registration date by ascending order for each customer
rp = rp.loc[:].sort_values(['EMAIL_ADDRESS_ID','REGISTRATION_DATE'], ascending=True)
rp.reset_index(drop=True, inplace=True)

In [ ]:
# Convert birth date column to caculatable type for creating age column.
rp['BIRTH_DATE'] = pd.to_datetime(rp['BIRTH_DATE'], format='%Y-%m-%d %H:%M:%S')

# Create age column
rp["age"] = pd.to_datetime('2020-10-09 00:00:00') - rp['BIRTH_DATE']

# Covert age column data type to integer.
rp['age'] = rp['age'].dt.days.astype('int64')

In [ ]:
# Because measurement of age value is in days, so we need to convert days to years
rp['age'] = (rp['age'] / 365).round(decimals=0)

# Take a look at how age distribute
print("AGE <15:", rp[rp["age"] < 15].shape[0])
print("AGE >=100 :", rp[rp["age"] >= 80].shape[0])

sns.boxplot(x=rp["age"]);

In [ ]:
# As we can see from boxplot of age, we can create a age group to clearly identify how customers distribute
bins = [0, 14, 24, 49, 64, 90, 1000]
labels = ['children', 'youth', 'young adult', 'senior adult', 'seniors', 'extreme old']
rp['age_group'] = pd.cut(rp['age'], bins=bins, labels=labels)

In [ ]:
# Check top 10 category of products in sale.
cate = rp.copy()
cate = pd.DataFrame(cate['GSCM_Prod_Name_sf'].value_counts().iloc[:10])
cate = cate.reset_index()
cate = cate.rename(columns={'GSCM_Prod_Name_sf': 'qty', 'index': 'category'})

ay = sns.barplot(y='category', x='qty', data=cate);

In [ ]:
# Check top 10 most popular colors.
color = rp.copy()
color = pd.DataFrame(color['Attb12_gscm'].value_counts().iloc[:10])
color = color.reset_index()
color = color.rename(columns={'Attb12_gscm': 'qty', 'index': 'color'})

axy = sns.barplot(y='color', x='qty', data=color);

In [ ]:
# Create a columns called color to differentiate popular colors from minor colors
rp['color'] = np.where(rp['Attb12_gscm'].isin(['BLACK', 'ORCHID GRAY', 'PURPLE', 'SILVER', 'WHITE']), \
                        rp['Attb12_gscm'], 'other')

In [ ]:
# Check out bset sell hhp models in RP table.
cat = rp.copy()
hhp = cat[cat['GSCM_Prod_Name_sf'] == 'HHP']['Attb09_gscm'].value_counts()

#Prepare dataframe for further chat plotted
hhp = pd.DataFrame(hhp)
hhp = hhp.reset_index().rename({'index': 'hhp_models'})

# Plot the chart.
hhp = hhp.rename(columns={'index': 'hhp_models', 'Attb09_gscm': 'qty'})
ax = sns.barplot(y='hhp_models', x='qty', data=hhp.iloc[:15]);

In [ ]:
# prepare a dataframe to create new column called number of registration
tp = rp.groupby('EMAIL_ADDRESS_ID')['GSCM_Prod_Name_sf'].count()
cpt = pd.DataFrame(data=tp)
cpt.reset_index(inplace=True)

# Merge two tables to add in one column
rp = rp.merge(cpt, on='EMAIL_ADDRESS_ID', how='left')

In [ ]:
# change column name to correct one
rp = rp.rename(columns={'GSCM_Prod_Name_sf_y':'Number_of_registration'})

In [ ]:
# In order to get lastest registration date for each customer
idx = rp['EMAIL_ADDRESS_ID'].drop_duplicates(keep='last').index
i = rp.loc[idx]

# to get number of days of last purchase of product
#i['num_of_days_since_last_pro_purchase'] = pd.to_datetime('2020-10-09 00:00:00') - i['REGISTRATION_DATE']

# #Get the new column of ''num_of_days_since_last_pro_purchase''
# ii = i[['EMAIL_ADDRESS_ID', 'num_of_days_since_last_pro_purchase']]
# rp = rp.merge(ii, on='EMAIL_ADDRESS_ID', how='left')

# # convert data tpe from timedelta to integert.
# rp['num_of_days_since_last_pro_purchase'] = rp['num_of_days_since_last_pro_purchase'].dt.days.astype('int32')

In [ ]:
# Create a column called FSA in order to adding buying capacity further
rp['FSA'] = rp['PostalCodeArea'].map(lambda x: x[0:3])

### Machine learning

In [ ]:
# Check if lable is imbalanced and result shows we need to use sampling tool to samplt the data.
df['Churn'].value_counts()
print('Percentage of Churn is ', df[df['Churn'] == 1]['Churn'].count() / df[['Churn']].count())

In [ ]:
df = df.drop(columns=['Unnamed: 0', 'City_PC', 'Prov_PC', 'age', 'since_last_open', 'FSA'])
# Redundant column generated when merging tables, drop it.

df = df.drop(columns=['EMAIL_ADDRESS_ID', 'REGISTRATION_DATE', 'GSCM_Prod_Name_sf_x', 'LastOpenedDate'])
# Drop Categorical columns which does not help model building.

# Made categorical columns switch to readable numerical columns for the models
df = pd.get_dummies(columns=['age_group', 'color', 'models'], data=df)

In [ ]:
# Check if we have missing values before build model.
df.isnull().sum()

In [ ]:
# Check the correlation Matrix
fig, ax = plt.subplots()
fig.set_size_inches(30, 25)
sns.heatmap(df.corr(), annot=True);

In [ ]:
# Baseline model XGBoost
X = df.drop(columns=['Churn']) # all features
y = df['Churn'] # label
# Splited data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# In order to avoid data leakage, imputed missing values of buying capacity.
mean_bc = X_train['buy_capacity'].mean()
# Filling NaN value with average of buying capacity for training and testing sets.
X_train = X_train.fillna(mean_bc)
X_test = X_test.fillna(mean_bc)

os = SMOTE()
X_train, y_train = os.fit_resample(X_train, y_train)

dt = XGBClassifier() # Applying estimator - DecisionTree classifier.
dt.fit(X_train, y_train) # Fitted the model with Training set.
y_pred = dt.predict(X_test) # Made prediction for test set by applying trained decisionTree model.

In [ ]:
y_train_pred = dt.predict(X_train)
print(classification_report(y_train_pred, y_train))
print(classification_report(y_test, y_pred))
# Printed the report of model performance.

In [ ]:
pd.Series(dt.feature_importances_, index=X.columns).sort_values().plot(kind='bar');

### As feature importance graph shoed above, OPT_OUT, Open and Clicks play important roles in the model.

In [ ]:
# Handle outliers - median of each columns will not be influenced significantly compared to average
def remove_outliers(x):
# Definite a function to replace outliers with medians of each column in training set
    X_train[x] = np.where(X_train[x] >= X_train[x].quantile(0.95), X_train[x].quantile(0.95), X_train[x])
    
# Replace outliers in test set with medians of each column in training set
    X_test[x] = np.where(X_test[x] >= X_train[x].quantile(0.95), X_train[x].quantile(0.95), X_test[x])

In [ ]:
X = df.drop(columns=['Churn']) # Created all features
y = df['Churn'] # Created label
# Splited data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Have a list of features that need to be handled outliers
lst = ['Number_of_registration', 'Open', 'Clicks']
for i in lst:
    remove_outliers(i)
    
# In order to avoid data leakage, imputed missing values of buying capacity.
mean_bc = X_train['buy_capacity'].mean()
# Filling NaN value with average of buying capacity for training and testing sets.
X_train = X_train.fillna(mean_bc)
X_test = X_test.fillna(mean_bc)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# LogisticRegression

ratio_lr = 0.5
os = SMOTE(sampling_strategy=ratio_lr)
X_train_os, y_train_os = os.fit_resample(X_train_scaled, y_train)

param = {
    #'penalty':['l1', 'l2']，
    'C': [1, 0.1, 0.01, 0.001]
}

lr = LogisticRegression(class_weight={0: ratio_lr, 1: 1 - ratio_lr})
gs_lr = GridSearchCV(lr, param, cv=3, scoring='f1', n_jobs=-1, verbose=2)
gs_lr.fit(X_train_os, y_train_os)

In [ ]:
# get best estimator and get predict proba for each prediction.
gsbm_lr = gs_lr.best_estimator_
y_train_pred_proba = gsbm_lr.predict_proba(X_train_scaled)[:, 1]
y_pred_proba = gsbm_lr.predict_proba(X_test_scaled)[:, 1]

# Set up threshold for determine "1" and "0" in order to tune the model.
thre = 0.7
y_train_pred_proba_lr = list(map(lambda x: 1 if x >= thre else 0, y_train_pred_proba))
y_pred_proba_lr = list(map(lambda x: 1 if x >= thre else 0, y_pred_proba))

print(classification_report(y_train, y_train_pred_proba_lr))
print(classification_report(y_test, y_pred_proba_lr))

In [ ]:
# Check different threholds affecting the results
print(pd.Series(y_pred_proba > 0.5).value_counts())
print(pd.Series(y_pred_proba > 0.8).value_counts())

In [ ]:
# plot ROC curve

def plot_roc(y_train, y_test, y_pred_prob, y_train_pred_prob):
    # Plot a ROC curve
    plt.figure(1, figsize=(8, 8))
    plt.plot([0, 1], [0, 1], 'k--')
    
    fpr_train, tpr_train, _ = roc_curve(y_train, y_train_pred_prob)
    fpr_test, tpr_test, _ = roc_curve(y_test, y_pred_prob)
    plt.plot(fpr_train, tpr_train, label='Training set')
    plt.plot(fpr_test, tpr_test, label='Test set')
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc='best')
    return

In [ ]:
plot_roc(y_train, y_test, y_pred_proba, y_train_pred_proba)

In [ ]:
# DecisionTree with scaler

ratio_dt = 0.5
os = SMOTE(sampling_strategy=ratio_dt)
X_train_os, y_train_os = os.fit_resample(X_train_scaled, y_train)

param = {
    #'criterion':['gini', 'entropy'],
    'max_depth': [10,20,30,40] 
}
dt = DecisionTreeClassifier()
gs_dt = GridSearchCV(dt, param, cv=3, scoring='f1', n_jobs=-1, verbose=2)
gs_dt.fit(X_train_scaled, y_train)

In [ ]:
# get best estimator and get predict proba for each prediction.
gsbm_dt = gs_dt.best_estimator_
y_train_pred_proba = gsbm_dt.predict_proba(X_train_scaled)[:, 1]
y_pred_proba = gsbm_dt.predict_proba(X_test_scaled)[:, 1]

# Set up threshold for determine "1" and "0" in order to tune the model.
thre = 0.5
y_train_pred_proba_dt = list(map(lambda x: 1 if x >= thre else 0, y_train_pred_proba))
y_pred_proba_dt = list(map(lambda x: 1 if x >= thre else 0, y_pred_proba))

print(classification_report(y_train, y_train_pred_proba_dt))
print(classification_report(y_test, y_pred_proba_dt))

In [ ]:
# RandomForest with scaler

ratio_rf = 0.5
os = SMOTE(sampling_strategy=ratio_rf)
X_train_os, y_train_os = os.fit_resample(X_train_scaled, y_train)

param = {
    #'criterion':['gini', 'entropy'],
    'max_depth':[10,20,30,40],
    'n_estimators':[100, 150]
    #'min_samples_split':[2,3,4,5,8,10],
    #'max_features':['sqrt', 'log2']
}
rf = RandomForestClassifier()
gs_rf = GridSearchCV(rf, param, cv=3, scoring='f1', n_jobs=-1, verbose=2)
gs_rf.fit(X_train_scaled, y_train)

In [ ]:
gsbm_rf = gs_rf.best_estimator_
y_train_pred_proba = gsbm_rf.predict_proba(X_train_scaled)[:, 1]
y_pred_proba = gsbm_rf.predict_proba(X_test_scaled)[:, 1]

# Set up threshold for determine "1" and "0" in order to tune the model.
thre = 0.8
y_train_pred_proba_rf = list(map(lambda x: 1 if x >= thre else 0, y_train_pred_proba))
y_pred_proba_rf = list(map(lambda x: 1 if x >= thre else 0, y_pred_proba))

print(classification_report(y_train, y_train_pred_proba_rf))
print(classification_report(y_test, y_pred_proba_rf))

In [ ]:
# XGBoost

ratio_xg = 0.5
os = SMOTE(sampling_strategy=ratio_xg)
X_train_os, y_train_os = os.fit_resample(X_train_scaled, y_train)

param = {
    'max_depth ':[10, 15],
    'learning_rate ':[0.1]
}

xg = XGBClassifier()

gs_xg = GridSearchCV(xg, param, cv=3, scoring='f1', n_jobs=-1, verbose=2)
gs_xg.fit(X_train_os, y_train_os)

In [ ]:
gsbm_xg = gs_xg.best_estimator_
y_train_pred_proba = gsbm_xg.predict_proba(X_train_scaled)[:, 1]
y_pred_proba = gsbm_xg.predict_proba(X_test_scaled)[:, 1]

# Set up threshold for determine "1" and "0" in order to tune the model.
thre = 0.6
y_train_pred_proba_xg = list(map(lambda x: 1 if x >= thre else 0, y_train_pred_proba))
y_pred_proba_xg = list(map(lambda x: 1 if x >= thre else 0, y_pred_proba))

print(classification_report(y_train, y_train_pred_proba_xg))
print(classification_report(y_test, y_pred_proba_xg))

### MLP Classifier and Keras Sequential

In [ ]:
# Neural Network - MLPClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
os = SMOTE()
X_train_os, y_train_os = os.fit_resample(X_train_scaled, y_train)
mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10, 10), max_iter=1000)
mlp.fit(X_train_os, y_train_os)

In [ ]:
pred_mlp = mlp.predict(X_test_scaled)
print(classification_report(y_test, pred_mlp))

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential

In [ ]:
seq = Sequential()
seq.add(Dense(12, input_dim=29, activation='relu'))
seq.add(Dense(9, activation='relu'))
seq.add(Dense(1, activation='sigmoid'))

seq.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

seq.fit(X_train_scaled, y_train, epochs=15, batch_size=10)

pred_seq = seq.predict_classes(X_test_scaled)

print(classification_report(y_test, pred_seq))

In [ ]:
print(y_train.shape)
print(X_train_scaled.shape)
pred_seq

### Plot ROC_curve for models

In [ ]:
X = df.drop(columns=['Churn']) 
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

lst = ['Number_of_registration', 'Open', 'Clicks']
for i in lst:
    remove_outliers(i)

mean_bc = X_train['buy_capacity'].mean()
X_train = X_train.fillna(mean_bc)
X_test = X_test.fillna(mean_bc)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

os = SMOTE(sampling_strategy=0.5)
X_train_os, y_train_os = os.fit_resample(X_train_scaled, y_train)

models = [
    {
    'label': 'Logistic Regression',
    'model': LogisticRegression(C=1)
    },
    {'label': 'Random Forest',
     'model': RandomForestClassifier(max_depth=20, n_estimators=150)
    },
    {'label': 'Decision Tree',
     'model': DecisionTreeClassifier(max_depth=20)},
    {'label': 'XGBoost',
     'model': XGBClassifier(learning_rate=0.1, max_depth=10)},
    {'label': 'K-Nearest Neighbor',
     'model': KNeighborsClassifier(weights='uniform', n_neighbors=5, algorithm='auto')},
    {'label': 'Gradient Boosting',
     'model': GradientBoostingClassifier(learning_rate=0.1)
    }
]
for i in models:
    model = i['model']
    model.fit(X_train_os, y_train_os)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, model.predict_proba(X_test_scaled)[:,1])
    auc = metrics.roc_auc_score(y_test, model.predict(X_test_scaled))
    plt.plot(fpr, tpr, label='%s ROC (area=%0.2f)' % (i['label'], auc))
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('sensitivity(true Positive Rate)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()